In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time
from datetime import datetime
from homeharvest import scrape_property
import matplotlib.pyplot as plt

for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

./Boston_HomeHarvest_20250910_122253.csv
./Project_2.ipynb
./.ipynb_checkpoints/Project_2-checkpoint.ipynb
./.ipynb_checkpoints/Boston_HomeHarvest_20250910_122253-checkpoint.csv


# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

In [30]:
"Predict home prices in your specific city or area "

'Predict home prices in your specific city or area '

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

In [31]:
# def get_sold_year(soldDate):
#     """soldDate may be 'YYYY-MM-DD' or epoch ms/sec; return year or None."""
#     if soldDate is None:
#         return None
#     try:
#         iv = int(soldDate)
#         # epoch ms vs s
#         if iv > 10**12:
#             return datetime.utcfromtimestamp(iv/1000.0).year
#         if iv > 10**9:
#             return datetime.utcfromtimestamp(iv).year
#     except Exception:
#         pass
#     try:
#         return datetime.fromisoformat(str(val)[:10]).year
#     except Exception:
#         return None

# def scrape_zillow_data(city='boston', state='ma', max_pages=24, pause=1.5):
#     """
#     Returns a DataFrame of mostly-raw fields + a derived 'Age' column.
#     """
#     headers = {
#         "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
#         "Accept-Language": "en",
#         "Cache-Control": "no-cache",
#         "Pragma": "no-cache",
#         "Sec-Ch-Ua": '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
#         "Sec-Ch-Ua-Mobile": "?0",
#         "Sec-Ch-Ua-Platform": '"Windows"',
#         "Sec-Fetch-Dest": "document",
#         "Sec-Fetch-Mode": "navigate",
#         "Sec-Fetch-Site": "none",
#         "Sec-Fetch-User": "?1",
#         "Upgrade-Insecure-Requests": "1",
#         "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
#     }
#     all_props, page_number = [], 1

#     while page_number <= max_pages:
#         url = f"https://www.zillow.com/{city}-{state}/sold/" if page_number == 1 else f"https://www.zillow.com/{city}-{state}/sold/{page_number}_p/"
#         print(f"[{city},{state}] page {page_number}: {url}")
#         try:
#             r = requests.get(url, headers=headers, timeout=25)
#             r.raise_for_status()
#         except requests.RequestException as e:
#             print(f"  request error on page {page_number}: {e}")
#             break

#         soup = BeautifulSoup(r.content, 'html.parser')
#         script_tag = soup.find('script', {'id': '__NEXT_DATA__'})
#         if not script_tag:
#             print("  __NEXT_DATA__ not found; structure may have changed or throttled.")
#             break

#         try:
#             jd = json.loads(script_tag.string)
#             results = jd['props']['pageProps']['searchPageState']['cat1']['searchResults']['listResults']
#             if not results:
#                 print("  no results on this page; stopping this city.")
#                 break

#             page_props = []
#             for item in results:
#                 hdp = item.get('hdpData', {}) or {}
#                 home = hdp.get('homeInfo', {}) or {}

#                 year_built = home.get('yearBuilt')
#                 sold_date_raw = item.get('soldDate') or home.get('dateSold')
#                 sold_year = get_sold_year(sold_date_raw)
#                 current_year = datetime.now().year

#                 age = None
                

#                 page_props.append({
#                     'City': city,
#                     'State': state,
#                     'Address': item.get('address', 'N/A'),
#                     'AIN': home.get('parcelId'),
#                     'Sold Price': item.get('soldPrice', item.get('price')),
#                     'Bedrooms': item.get('beds'),
#                     'Bathrooms': item.get('baths'),
#                     'Area (SqFt)': item.get('area'),
#                     'Property Type': home.get('homeType', 'N/A'),
#                     'Year Built': home.get('yearBuilt'),
#                     'Age': age,                       # derived (post-capture)
#                     'Sold Date (raw)': sold_date_raw, # keep raw for audit/2yr filter later
#                     'Lot Size (val)': home.get('lotAreaValue'),
#                     'Lot Size (unit)': home.get('lotAreaUnit'),
#                     'HOA Fee': home.get('hoaFee'),
#                     'Latitude': (item.get('latLong') or {}).get('latitude'),
#                     'Longitude': (item.get('latLong') or {}).get('longitude'),
#                     'ZPID': item.get('zpid') or home.get('zpid'),
#                     'DetailURL' : item.get('detailUrl') or item.get('hdpUrl')
#                 })

#             all_props.extend(page_props)
#             print(f"  +{len(page_props)} this page (total {len(all_props)})")
#             page_number += 1
#             time.sleep(pause)

#         except (KeyError, json.JSONDecodeError) as e:
#             print(f"  parse error: {e}")
#             break

#     return pd.DataFrame(all_props)

# def full_scrape(city_state, target_rows=1500, max_pages_per_city=200, pause=2.0):
#     frames, total = [], 0
#     for (city, state) in city_state:
#         if total >= target_rows:
#             break
#         df = scrape_zillow_data(city=city, 
#                                 state=state, 
#                                 max_pages=max_pages_per_city, 
#                                 pause=pause)
#         if not df.empty:
#             frames.append(df)
#             total += len(df)
#             print(f"Accumulate: {total} rows so far")
#         time.sleep(pause * 1.5)
#     if frames:
#         out = pd.concat(frames, ignore_index=True)
#         # Remove repeats
#         out.drop_duplicates(subset=['Address','Sold Price','Year Built','Area (SqFt)'], inplace=True)
#         return out
#     return pd.DataFrame()

# if __name__ == "__main__":
#     # Greater Boston area, gets just under 3k
#     Cities = [
#         ('boston','ma'),
#         ('cambridge','ma'),
#         ('somerville','ma')
#     ]

#     df = full_scrape(Cities, target_rows=2000, max_pages_per_city=250, pause=2.0)

#     if not df.empty:

#         df_links = df[['Address', 'DetailURL', 'ZPID']].copy()
#         df = df.drop(columns=['DetailURL'])
    
#         two_years_ago = datetime.now().year - 2
#         df['_sold_year'] = df['Sold Date (raw)'].apply(get_sold_year)
#         df2 = df[df['_sold_year'].fillna(0) >= two_years_ago].drop(columns=['_sold_year'])
#         df2.to_csv('zillow_boston.csv', index=False)
#     else:
#         print("\nNo data scraped.")

In [40]:
"""
My Process to gather data
"""

# current_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# filename = f"Boston_HomeHarvest_{current_timestamp}.csv"

# properties = scrape_property(
#   location="Boston, MA",
#   listing_type="sold",  # or (for_sale, for_rent, pending)
#   past_days=180

  # property_type=['single_family','multi_family'],
  # date_from="2023-05-01", # alternative to past_days
  # date_to="2023-05-28",
  # foreclosure=True
  # mls_only=True,  # only fetch MLS listings
# )
# print(f"Number of properties: {len(properties)}")

# Export to csv
# properties.to_csv(filename, index=False)
# print(properties.head())

data = pd.read_csv("Boston_HomeHarvest_20250910_122253.csv")
data.head()

,property_url,property_id,listing_id,permalink,mls,mls_id,status,mls_status,text,style,...,builder_id,builder_name,office_id,office_mls_set,office_name,office_email,office_phones,nearby_schools,primary_photo,alt_photos
0,https://www.realtor.com/realestateandhomes-det...,9990188513,2.980653e+09,65-Lewis-St-Apt-306_Boston_MA_02128_M99901-88513,BSMA,73360418,SOLD,Sold,Welcome Home to Luxury Waterfront Living At It...,CONDOS,...,NaN,NaN,2021231.0,O-BSMA-AN0297,Urban Circle Realty,sherean@urbancirclerealty.com,"[{'number': '6179030371', 'type': 'Office', 'p...","Excel Academy Charter (District), Boston Publi...",https://ap.rdcpix.com/f50367ae4790d53b226cef69...,https://ap.rdcpix.com/f50367ae4790d53b226cef69...
1,https://www.realtor.com/realestateandhomes-det...,9809784357,2.982669e+09,293-295-Commonwealth-Ave-Unit-3C_Boston_MA_021...,BSMA,73384746,SOLD,Sold,Nestled in a professionally managed brownstone...,CONDOS,...,NaN,NaN,2935107.0,O-BSMA-AN2213,Berkshire Hathaway Homeservices Warren Residen...,nwarren@warrenre.com,"[{'number': '6178489616', 'type': 'Office', 'p...",Boston Public Schools,https://ap.rdcpix.com/2f5dca696f84373398d15490...,https://ap.rdcpix.com/2f5dca696f84373398d15490...
2,https://www.realtor.com/realestateandhomes-det...,9542510902,2.984848e+09,65-Magnolia-St_Boston_MA_02125_M95425-10902,BSMA,73411594,SOLD,Sold,Beautiful Grand Colonial on a Corner Lot! This...,SINGLE_FAMILY,...,NaN,NaN,NaN,O-BSMA-AN7113,"Hadassah Homes, Inc.",NaN,"[{'number': '7817763677', 'type': None, 'prima...","Boston Collegiate Charter (District), Conserva...",https://ap.rdcpix.com/f9a9728eff9ada303e20ba4e...,https://ap.rdcpix.com/f9a9728eff9ada303e20ba4e...
3,https://www.realtor.com/realestateandhomes-det...,4001051816,2.984817e+09,6-Fairfax-St_Boston_MA_02124_M40010-51816,BSMA,73410954,SOLD,Sold,Between Peabody Sq & Adams Village off Carruth...,SINGLE_FAMILY,...,NaN,NaN,1508964.0,O-BSMA-AC1396,William Raveis Real Estate - Boston - Back Bay,Arlington@raveis.com,"[{'number': '6172665200', 'type': 'Office', 'p...","Codman Academy Charter Public (District), Neig...",https://ap.rdcpix.com/e7531bd1c94f8787fa8574ce...,https://ap.rdcpix.com/e7531bd1c94f8787fa8574ce...
4,https://www.realtor.com/realestateandhomes-det...,3928122366,2.981419e+09,519-Harrison-Ave-Apt-D212_Boston_MA_02118_M392...,BSMA,73369448,SOLD,Sold,DND affordable unit with eligibility requireme...,CONDOS,...,NaN,NaN,2997232.0,O-BSMA-AN3337,COMPASS,info@compass.com,"[{'number': '6172063333', 'type': 'Mobile', 'p...",Boston Public Schools,https://ap.rdcpix.com/ce84f87d920c171461aedea0...,https://ap.rdcpix.com/ce84f87d920c171461aedea0...


# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

In [48]:
# Example data: Square footage and house prices
import numpy as np
sq_footage =  data.dropna(subset="sqft")
prices_scatter = 150 * sq_footage + np.random.normal(0, 50000, 50) # Simulate a relationship

plt.figure(figsize=(10, 6))
plt.scatter(sq_footage, prices_scatter, alpha=0.7)
plt.title('House Price vs. Square Footage')
plt.xlabel('Square Footage')
plt.ylabel('Price ($)')
plt.grid(True)
plt.show()

ValueError: Unable to coerce to Series, length must be 64: given 50

# 4.Prepare the Data


Apply any data transformations and explain what and why


In [15]:
def price_to_numeric(price_str):
    # Changes Price to integer value
    if price_str is None:
        return None
    if isinstance(price_str, (int, float)):
        return int(price_str)
    s = str(price_str).strip().upper().replace('$','').replace(',','')
    if 'M' in s:
        return int(float(s.replace('M','')) * 1_000_000)
    if 'K' in s:
        return int(float(s.replace('K','')) * 1_000)
    try:
        return int(float(s))
    except Exception:
        return None


# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [ ]:
def infrence(prams):
    results = m.run(prams)
    return results 